In [1]:
import Watchmen

watchmen = Watchmen.Watchmen(youtube_api_key_path='E:\\keys\\youtubeDataApiKey.txt',openai_api_key_path='E:\\keys\\openAIApiKey.txt', spacy_model='en_core_web_trf', refresh_indexes=False)

setting up librarian...
setting up youtube explorer...
setting up scholar...


In [54]:
df = watchmen.librarian.collect_multi_entity_data(['trump'])
df = df[df['region']=='USA']
df.head()

,lib_key,source_id,rec_mod_dtm,doc_type_id,source_system_id,path,summary,summary_reasoning,sentiment,primary_emotion,secondary_emotion,channel_id,channel_title,category,region,entity
0,2077,0dYFFxBTbhg,2025-01-01 17:11:39,2,2,./data/transcripts/0dYFFxBTbhg_transcript.parquet,The Senate Judiciary Committee released a repo...,The document discusses a report released by th...,True,anger,exasperation,UCaXkIU1QidjPwiAYu6GcHjg,NBC News,News,USA,trump
0,2085,18gjwzCbN_A,2025-01-01 17:11:11,2,2,./data/transcripts/18gjwzCbN_A_transcript.parquet,The document analyzes Mike Johnson's bid for S...,The document discusses the political dynamics ...,True,surprise,exasperation,UCaXkIU1QidjPwiAYu6GcHjg,NBC News,News,USA,trump
0,2930,2Jxa76aSHK4,2025-01-08 19:08:55,2,2,./data/transcripts/2Jxa76aSHK4_transcript.parquet,The document features a political figure refle...,The document discusses a conversation with a p...,True,love,pride,UCP6HGa63sBC7-KHtkme-p-g,USA Today,News,USA,trump
0,2098,2yI8UTm0fuQ,2025-01-01 17:11:00,2,2,./data/transcripts/2yI8UTm0fuQ_transcript.parquet,The document outlines President Biden's transi...,The document discusses President Biden's trans...,True,joy,optimism,UCaXkIU1QidjPwiAYu6GcHjg,NBC News,News,USA,trump
0,2728,2zaUEUAYETE,2025-01-04 14:47:45,2,2,./data/transcripts/2zaUEUAYETE_transcript.parquet,The speech expresses gratitude for a nominatio...,The document is a speech delivered by a politi...,True,joy,optimism,UCaXkIU1QidjPwiAYu6GcHjg,NBC News,News,USA,trump


In [55]:
def split_emot(emot):
    if type(emot) is list:
        return emot[0]
    else:
        return emot

dfchan = df.groupby(['channel_title']).lib_key.count().reset_index().rename(columns={'lib_key':'channel_ttl'})
df['primary_emotion'] = df['primary_emotion'].apply(lambda x: split_emot(x))
dfa = df.groupby(['region','entity','channel_title','primary_emotion','secondary_emotion','sentiment']).lib_key.count().reset_index().rename(columns={'lib_key':'videos'})
dfa = dfa.join(dfchan.set_index('channel_title'), on='channel_title')
dfa['perc'] = dfa['videos'] / dfa['channel_ttl']
dfa = dfa[dfa['primary_emotion']!='None']
dfa.head()

,region,entity,channel_title,primary_emotion,secondary_emotion,sentiment,videos,channel_ttl,perc
0,USA,trump,Associated Press,anger,exasperation,False,4,20,0.20
1,USA,trump,Associated Press,anger,horror,False,1,20,0.05
2,USA,trump,Associated Press,anger,optimism,True,1,20,0.05
3,USA,trump,Associated Press,fear,horror,False,4,20,0.20
4,USA,trump,Associated Press,joy,optimism,True,1,20,0.05


In [56]:
import plotly.express as px

dfUSA = dfa[dfa['region']=='USA']
fig = px.bar(dfa[(dfa['region']=='USA') & (dfa['entity']=='trump')], x='secondary_emotion', y='perc', color='sentiment', facet_col='channel_title')
fig.show()